[IPython Notebook](general_concepts.ipynb) |  [Python Script](general_concepts.py)

General Concepts
======================

This tutorial introduces all the general concepts of dealing with the Bundle, ParameterSets, and Parameters.  This tutorial aims to be quite complete - covering almost everything you can do with Parameters, so on first read you may just want to try to get familiar, and then return here as a reference for any details later.

All of these tutorials assume basic comfort with Python in general - particularly with the concepts of lists, dictionaries, and objects as well as basic comfort with using the numpy and matplotlib packages.

Setup
----------------------------------------------

Let's get started with some basic imports

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


If running in IPython notebooks, you may see a "ShimWarning" depending on the version of Jupyter you are using - this is safe to ignore.

PHOEBE 2 uses constants defined in the IAU 2015 Resolution which conflict with the constants defined in astropy.  As a result, you'll see the warnings as phoebe.u and phoebe.c "hijacks" the values in astropy.units and astropy.constants.

Whenever providing units, please make sure to use the phoebe.u instead of astropy.units.

### Logger

Before starting any script, it is a good habit to initialize a logger and define which levels of information you want printed to the command line (clevel) and dumped to a file (flevel).

The levels from most to least information are:

* DEBUG
* INFO
* WARNING
* ERROR
* CRITICAL


In [2]:
logger = phoebe.logger(clevel='INFO', flevel='DEBUG', filename='tutorial.log')

All of these arguments are optional and will default to clevel='WARNING' if not provided.

So with this logger, anything with INFO, WARNING, ERROR, or CRITICAL levels will be printed to the screen.  All messages of any level will be written to a file named 'tutorial.log' in the current directory.

Note: the logger messages are not included in the outputs shown below.

Bundle and ParameterSets
----------------------------------

### The Bundle

The Bundle is a python object which holds all information about a system and its datasets, models, and fitting runs.  Below you'll see how to access and set individual parameters within the bundle as well as how to filter its contents.

The Bundle is just a glorified python dictionary of parameters (making it a 'ParameterSet') but with very flexible keys and the added ability to filter.

Let's start by initializing an empty Bundle:

In [3]:
b = phoebe.Bundle()
b

<PHOEBE Bundle: 11 parameters | contexts: setting, system>

Here we can see that the Bundle already has a number of parameters, and that we could further filter these results by providing one of the available 'contexts' - more on that in just a bit.

With the exception of some of the available bundle-specific methods, which we'll also see later, the bundle itself is really just the all-inclusive top-level ParameterSet.  So let's continue by talking about ParameterSets in general.

### ParameterSets

Let's first look at the different methods for viewing all the information in a given ParameterSet.  In this case we'll start with the entire Bundle - remember, the Bundle is a ParameterSet.

In [4]:
print b


SYSTEM:
distance
hierarchy
t0
vgamma
epoch
ra
dec

COMPONENT:


DATASET:


CONSTRAINT:


COMPUTE:


MODEL:


FITTING:


FEEDBACK:


PLUGIN:





*Note: this output is currently very ugly and will be formatted to be more user-friendly in the near future*

In [5]:
b.to_list()

[<Parameter: t0=0.0 d | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>,
 <Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>,
 <Parameter: dec=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>,
 <Parameter: epoch=J2000 | keys: description, value, relevant_if>,
 <Parameter: distance=10.0 pc | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>,
 <Parameter: vgamma=0.0 km / s | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>,
 <Parameter: log_history=False | keys: description, value, relevant_if, copy_for>,
 <Parameter: dict_filter={} | keys: description, value, relevant_if, copy_for>,
 <Parameter: dict_set_all=False | keys: description, value, relevant_if, copy_for>,
 <Parameter: plotting_backend=mpl | keys: description, choices, value, relevant

Here we can see that a ParameterSet is really just a collection of Parameters.  

Each of these parameters has several "metatags".  Let's just take the first parameter from this list.  Shortly we'll see nicer ways to get a particular Parameter from a ParameterSet, but for now we'll just pull the first one from the list.

In [6]:
param = b.to_list()[0]
param.meta

OrderedDict([('time', None),
             ('qualifier', 't0'),
             ('history', None),
             ('component', None),
             ('dataset', None),
             ('constraint', None),
             ('compute', None),
             ('model', None),
             ('fitting', None),
             ('feedback', None),
             ('plugin', None),
             ('method', None),
             ('context', 'system'),
             ('twig', 't0@system'),
             ('uniquetwig', 't0@system')])

Most of these "metatags" act as labels - for example, you can give a component tag to each of the components for easier referencing.

But a few of these tags are fixed and not editable:

* qualifier: literally the name of the parameter.
* method: tells what structures a parameter belongs to (ie whether a component is a star or an orbit).
* context: tells what context this parameter belongs to
* twig: a shortcut to the parameter in a single string.
* uniquetwig: the minimal twig needed to reach this parameter.
* uniqueid: an internal representation used to reach this parameter

These contexts are (you'll notice that most are represented in the tags):

* setting
* history
* system
* component
* dataset
* constraint
* compute
* model
* fitting
* feedback
* plugin

One way to distinguish between context and method is with the following question and answer:

"What kind of **[context]** is this?  It's a **[method]** tagged **[context]**=**[tag-with-same-name-as-context]**."

In different cases, this will then become:

* "What kind of **component** is this?  It's a **star** tagged **component**=**starA**." (context='component', method='star', component='starA')
* "What kind of **dataset** is this?  It's a **LC (light curve)** tagged **dataset**=**lc01**." (context='dataset', method='LC', dataset='lc01')
* "What kind of **compute** (options) are these?  They're **phoebe** (compute options) tagged **compute**=**preview**." (context='compute', method='phoebe', compute='preview')

Any of these tags can be accessed directly, through either a single parameter:


In [7]:
param.qualifier

't0'

... or at the ParameterSet level:

In [8]:
b.qualifier

This returns None since not all objects in this ParameterSet (the Bundle) share a single context.  But you can see all the options for a given key by providing the plural version of that tag name:

In [9]:
b.qualifiers

['distance',
 'dict_set_all',
 'plotting_backend',
 't0',
 'vgamma',
 'hierarchy',
 'epoch',
 'ra',
 'dict_filter',
 'log_history',
 'dec']

### Filtering ParameterSets

Any of the tags can also be used to filter the bundle:

In [10]:
b.filter(context='system')

<ParameterSet: 7 parameters | qualifiers: distance, hierarchy, t0, vgamma, epoch, ra, dec>

Here we were returned a ParameterSet of all Parameters that matched the filter criteria.  We can do almost anything to a ParameterSet that we could do to the bundle... including further filtering.  You can see that the representation of the ParameterSet now tells us a list of qualifiers instead of a list of contexts - it always shows the highest level of those "metatags" in which specifying a value will further limit the search.  In this case, all of these Parameters share identical "metatags" up to the qualifier.

So let's chain another filter call onto the end of our previous filter:

In [11]:
b.filter(context='system').filter(qualifier='ra')

<ParameterSet: 1 parameters>

Now we see that we have drilled down to a single Parameter.  Note that a ParameterSet is still returned - filter will *always* return a ParameterSet.

We could have accomplished the exact same thing with a single call to filter:

In [12]:
b.filter(context='system', qualifier='ra')

<ParameterSet: 1 parameters>

If you want to access the actual Parameter, you must use get instead of (or in addition to) filter.  All of the following lines do the exact same thing:

In [13]:
b.filter(context='system', qualifier='ra').get()

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [14]:
b.get(context='system', qualifier='ra')

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

Or we can use those twigs.  Twigs are just a combination of "metatags" separated by some delimiter.  You can use these for dictionary access in a ParameterSet - without needing to provide the name of the tag, and without having to worry about order.  Again, all of the following lines will do the exact same thing:

In [15]:
b['ra@system']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [16]:
b['system']['ra']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

You may notice that this returned a Parameter without needing to specify.  Twig dictionary access tries to be smart - if exactly 1 Parameter is found, it will return that Parameter instead of a ParameterSet.  Notice the difference between the two following lines:

In [17]:
b['system']

<ParameterSet: 7 parameters | qualifiers: distance, hierarchy, t0, vgamma, epoch, ra, dec>

In [18]:
b['ra@system']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

You can also access one more level - these keys that are listed in a parameter give you access to "attributes" of that Parameter:

In [19]:
print b['value@ra@system']

0.0


In [20]:
print b['description@ra@system']

Right ascension


Parameters
------------------------

Parameters are a single entry in a ParameterSet or Bundle.  They contain a value of some sort and any necessary information about that value (such as description or units, when applicable).

Let's first get a Parameter:

In [21]:
b['ra@system']

<Parameter: ra=0.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

As with filtering ParameterSets, these are accessible through either dictionary access or methods:

In [22]:
print b['value@ra@system']

0.0


In [23]:
print b['ra@system']['value']

0.0


In [24]:
print b['ra@system'].get_value()

0.0


In [25]:
print b.get_value('ra@system')

0.0


All Parameters also have a description that explains what the qualifier of the Parameter means.

In [26]:
print b['ra@system']['description']

Right ascension


Setting the value of a Parameter can be done through either the set_value method of the Bundle, or via dictionary access:

In [27]:
b['ra@system'] = 15
b['ra@system']

<Parameter: ra=15.0 deg | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [28]:
b['value@ra@system'] = 30
print b['value@ra@system']

30.0


In [29]:
b['ra@system'].set_value(45)
print b['ra@system'].get_value()

45.0


In [30]:
b.set_value('ra@system', 60)
print b.get_value('ra@system')

60.0


Parameters have different types - depending on what type of information they're supposed to hold.  We'll now take a look at the different types.  If you need to see the type of a Parameter, you can always request it:

In [31]:
type(b['ra@system'])

phoebe.parameters.parameters.FloatParameter

### Float Parameters

The most common type of Parameter holds a float value (and therefore a default unit to handle any type of necessary conversions).

You can access the default unit:

In [32]:
print b['ra@system']['default_unit']

deg


Request the value be converted to some other units:

In [33]:
print b.get_value('ra@system', 'rad')

1.0471975512


In [34]:
print b.get_value('ra@system', u.rad)

1.0471975512


Provide the value in some other units and have it converted for you by either providing an Quantity object (scalar times a unit object) or a tuple including the string representation of the unit:

In [35]:
b['ra@system'] = 1.2 * u.rad
print b['value@ra@system']

68.7549354157


In [36]:
b['ra@system'] = 1.4, 'rad'
print b['value@ra@system']

80.2140913183


Or even change the default units (for both setting and getting values):

In [37]:
b['default_unit@ra@system'] = u.rad
print b['value@ra@system']

1.4


In [38]:
b['ra@system'] = 1.5
print b['value@ra@system']

1.5


To get a Parameter's 'quantity' object that shows the units and allows converting after retrieving, use get_quanity instead:

In [39]:
print b.get_quantity('ra@system')

1.5 rad


In [40]:
print b.get_quantity('ra@system').to(u.deg)

85.9436692696 deg


In [41]:
print b.get_quantity('ra@system').to(u.deg).value

85.9436692696


### Int Parameters

Since we don't have any integer parameters yet, let's add some compute options.  Compute options will be discussed in much greater detail in the [Compute](compute) tutorial

In [42]:
b.add_compute(compute='mycompute')

/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


<ParameterSet: 19 parameters | qualifiers: dynamics_method, eclipse_alg, gridsize, stepsize, lc_method, atm, enabled, ltte, rv_grav, orbiterror, rv_method, etv_method, maxpoints, delta, etv_tol, store_mesh, distortion_method, subdiv_num, mesh_method>

In [43]:
print type(b['maxpoints@mycompute'])

<class 'phoebe.parameters.parameters.IntParameter'>


Integer Parameters are the same as float parameters, except they (obviously) accept integers instead of decimal numbers and they do not have units.

In [44]:
b['maxpoints@mycompute']

<Parameter: maxpoints=100000 | keys: description, value, limits, relevant_if, copy_for>

In [45]:
b['maxpoints@mycompute'] = 1000
b['maxpoints@mycompute']

<Parameter: maxpoints=1000 | keys: description, value, limits, relevant_if, copy_for>

Note that if you do provide a float, it will be casted to an integer.

In [46]:
b['maxpoints@mycompute'] = 100.5
b['maxpoints@mycompute']

<Parameter: maxpoints=100 | keys: description, value, limits, relevant_if, copy_for>

### Bool Parameters

Boolean Parameters are even simpler - they accept True or False.

The 'ltte' parameter is a switch in the compute options that tells the backend whether or not to account for ltte (light travel time effect) in the model.

In [47]:
print type(b['ltte@mycompute'])

<class 'phoebe.parameters.parameters.BoolParameter'>


In [48]:
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

In [49]:
b['ltte@mycompute'] = True
b['ltte@mycompute']

<Parameter: ltte=True | keys: description, value, relevant_if, copy_for>

Note that, like Int Parameters, Boolean Parameters will attempt to cast anything you give it into True or False.

In [50]:
b['ltte@mycompute'] = 0
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

In [51]:
b['ltte@mycompute'] = None
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

As with python, an empty string will cast to False and a non-empty string will cast to True

In [52]:
b['ltte@mycompute'] = ''
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

In [53]:
b['ltte@mycompute'] = 'somestring'
b['ltte@mycompute']

<Parameter: ltte=True | keys: description, value, relevant_if, copy_for>

The only exception to this is that (unlike Python), 'true' or 'True' will cast to True and 'false' or 'False' will cast to False.

In [54]:
b['ltte@mycompute'] = 'False'
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

In [55]:
b['ltte@mycompute'] = 'false'
b['ltte@mycompute']

<Parameter: ltte=False | keys: description, value, relevant_if, copy_for>

### String Parameters

String Parameters accept any valid string.

In [56]:
print type(b['epoch@system'])

<class 'phoebe.parameters.parameters.StringParameter'>


In [57]:
b['epoch@system']

<Parameter: epoch=J2000 | keys: description, value, relevant_if>

In [58]:
b['epoch@system'] = 'J9999'
b['epoch@system']

<Parameter: epoch=J9999 | keys: description, value, relevant_if>

### Choice Parameters

Choice Parameters act the same as String Parameters except that there are a limited number of valid strings accepted.

In [59]:
type(b['dynamics_method@mycompute'])

phoebe.parameters.parameters.ChoiceParameter

In [60]:
b['dynamics_method@mycompute']

<Parameter: dynamics_method=keplerian | keys: description, choices, value, relevant_if, copy_for>

In [61]:
b['dynamics_method@mycompute']['choices']

['keplerian', 'nbody']

In [62]:
b['dynamics_method@mycompute'] = 'nbody'
b['dynamics_method@mycompute']

<Parameter: dynamics_method=nbody | keys: description, choices, value, relevant_if, copy_for>

Trying to provide a string which is not in 'choices' will raise an error.

Relevant If
-------------------

Some parameters are only relevant depending on the value of another parameter.

In [63]:
b['mycompute']

<ParameterSet: 17 parameters | qualifiers: dynamics_method, eclipse_alg, store_mesh, stepsize, lc_method, atm, ltte, rv_grav, orbiterror, rv_method, etv_method, maxpoints, delta, etv_tol, distortion_method, subdiv_num, mesh_method>

In [64]:
b['delta']

<Parameter: delta=0.1 | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [65]:
b['delta']=0.2
b['delta']

<Parameter: delta=0.2 | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

In [66]:
b['delta']['relevant_if']

'mesh_method:marching'

In [67]:
b['mesh_method'] = 'wd'

In [68]:
b['mycompute']

<ParameterSet: 15 parameters | qualifiers: dynamics_method, eclipse_alg, gridsize, stepsize, lc_method, atm, ltte, rv_grav, orbiterror, rv_method, etv_method, etv_tol, store_mesh, mesh_method, subdiv_num>

Now that mesh_method=='wd', the delta parameter is no longer relevant, and so it's temporarily hidden.

In [69]:
b['delta']

<ParameterSet: EMPTY>

However, the value is still remembered as soon as it becomes relevant again

In [70]:
b['mesh_method'] = 'marching'
b['delta']

<Parameter: delta=0.2 | keys: description, value, quantity, default_unit, limits, relevant_if, copy_for, timederiv>

Copy For
--------------

Parameters also have a copy_for property that tells PHOEBE how to copy the Parameter if new components or datasets are added to the Bundle.

We'll see this in action later in the [Datasets Tutorial](./datasets), but for now, just know that you can *view* this property only and cannot change it.

In [71]:
b['mesh_method'].copy_for

{'component': '*', 'method': ['star', 'envelope']}

In this case, we see that the mesh_method will be *copied* for each new component that is added to the Bundle, so long as that component matches the filter in the dictionary (ie that it has a method of either 'star' or 'envelope').

You may never even need to view this property, but know that it exists and determines how Parameters may appear to be "duplicated" when adding datasets or components.